In [30]:
import json
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
from pprint import pprint
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
import matplotlib.pyplot as plt
import math
import numpy as np
from collections import Counter
import re, csv

arrange_pull_request_path = "/mnt/ext-hdd1/pull_request_npm_asia_2020/asia_arrange_pull_request_data" #updated 220301

from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from emoji import UNICODE_EMOJI

def is_emoji(s):
    return s in UNICODE_EMOJI

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/vittunyuta-m/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vittunyuta-m/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/vittunyuta-m/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [157]:
critical = ['breaking', 'change','high', 'density', 'break', 'security', 'risk', 'risky','attention',
            'severity', 'memory', 'leak',"blocked",'exclamation','failed', 'priority', 'blocker','denied',
            'urgent','critical','important','inconvenient','broken','required','needed','confusing','failing',
           'difficult','difficulty','hard','prio','performance',
           "securities", "incompatible", "hold", "effort", "followup","notable","must"]
non_critical = ["low",'minor','easy', 'no', 'non',
               "review",'medium']
lemmatizer = WordNetLemmatizer()
print(', '.join(sorted(critical)))
print(', '.join(sorted(non_critical)))

attention, blocked, blocker, break, breaking, broken, change, confusing, critical, denied, density, difficult, difficulty, effort, exclamation, failed, failing, followup, hard, high, hold, important, incompatible, inconvenient, leak, memory, must, needed, notable, performance, prio, priority, required, risk, risky, securities, security, severity, urgent
easy, low, medium, minor, no, non, review


In [54]:
stop_words = set(stopwords.words('english'))
additional_set = {'due','need', 'type','cat'}
stop_words.update(additional_set)
stop_words.remove('no')
# stop_words

## Main ========

In [55]:
with open("ranking_label_external.csv", mode='r') as infile:
    reader = csv.reader(infile)
    ex_label_dict = {rows[0]:rows[1] for idx, rows in enumerate(reader) if idx != 0}
    
with open("ranking_label_internal.csv", mode='r') as infile:
    reader = csv.reader(infile)
    in_label_dict = {rows[0]:rows[1] for idx, rows in enumerate(reader) if idx != 0}
    
with open("ranking_label_bot.csv", mode='r') as infile:
    reader = csv.reader(infile)
    bot_label_dict = {rows[0]:rows[1] for idx, rows in enumerate(reader) if idx != 0}

dataset = {
    "External": ex_label_dict,
    "Internal": in_label_dict,
    "Bot": bot_label_dict
}

In [79]:
all_labels = set(ex_label_dict.keys())
all_labels.update(set(in_label_dict.keys()))
all_labels.update(set(bot_label_dict.keys()))
len(all_labels)

2840

In [133]:
def findCriticalLabels(label_dict_with_counts):
    global length_count
    length_label_list = {}
    critical_labels = {}
    for label, count in label_dict_with_counts.items():
        cleaned_label = re.sub(r'[\(\)\u263a-\U0001f645\!]+',"", re.sub(r'[\s\:\-/\.]+'," ", label)).strip().lower()
        tokens = word_tokenize(cleaned_label)
        filtered_words = [w for w in tokens if not w.lower() in stop_words]
        lemmatized_output = [lemmatizer.lemmatize(w) for w in filtered_words]
        result = [w for w in lemmatized_output if not w.lower() in stop_words]
        #remove digit
        result = [x for x in result if not all(c.isdigit() for c in x)]
        c = True if [x for x in result if x in critical] else False
        if c:
            if not [x for x in result if x in non_critical]:
#                 print(label, result)
                critical_labels[label] = count
        
        if len(result) not in length_count:
            length_count[len(result)] = 0
        length_count[len(result)] += 1
        
        if len(result) not in length_label_list:
            length_label_list[len(result)] = []
        length_label_list[len(result)].append(label)
    return critical_labels, length_label_list

In [134]:
critical_label_dataset = {}
length_count = {}
length_group_list = {}
for group, group_label_dict in dataset.items():
    critical_label_dataset[group], length_group_list[group] = findCriticalLabels(group_label_dict)

In [116]:
for group, ddict in length_group_list.items():
    print(group)
    for le, label_list in ddict.items():
        print(le, len(label_list))

External
1 669
2 854
3 198
4 43
0 10
5 5
6 1
Internal
1 767
2 969
3 197
4 41
5 8
0 18
Bot
1 130
4 12
2 115
3 21
0 1


In [107]:
length_count

{1: 1566, 2: 1938, 3: 416, 4: 96, 0: 29, 5: 13, 6: 1}

In [158]:
import ast
ex_pr_df = pd.read_csv("external_PR.csv", converters={"labels":ast.literal_eval})
in_pr_df = pd.read_csv("internal_PR.csv", converters={"labels":ast.literal_eval})
bot_pr_df = pd.read_csv("bot_PR.csv", converters={"labels":ast.literal_eval})

pr_dataset = {
    "External": ex_pr_df,
    "Internal": in_pr_df,
    "Bot": bot_pr_df
}   

In [162]:
critical_pr_count = {
    "External": 0,
    "Internal": 0,
    "Bot": 0
}  
for pr_group, pr_df in pr_dataset.items():
    print(pr_group)
    current_group_critical_label_list = list(critical_label_dataset[pr_group].keys())
    for _, row in tqdm(pr_df.iterrows(), total=pr_df.shape[0]):
        isPRCritical = False
        for la in row["labels"]:
            if la in current_group_critical_label_list:
                isPRCritical = True
                break
        if isPRCritical:
            critical_pr_count[pr_group] += 1
#             print(row["labels"])
critical_pr_count

  0%|          | 1/290552 [00:00<8:26:44,  9.56it/s]

External


  0%|          | 1/340465 [00:00<15:36:34,  6.06it/s]

Internal


  0%|          | 1/314274 [00:00<12:58:35,  6.73it/s]

Bot


100%|██████████| 314274/314274 [00:16<00:00, 19591.60it/s]


{'External': 984, 'Internal': 2177, 'Bot': 3392}

In [164]:
all_critical_labels = None
for _, label_d in critical_label_dataset.items():
    if not all_critical_labels:
        all_critical_labels = set(label_d.keys())
    else:
        all_critical_labels.update(set(label_d.keys()))

all_count = len(all_critical_labels)
print("All:", all_count)
ex_count = len(critical_label_dataset["External"])
ex_top5 = dict(sorted(critical_label_dataset["External"].items(), key=lambda item: int(item[1]), reverse=True))
print("External: {} {:.2f}%".format(ex_count, ex_count/all_count*100))
print("External top 5:\n",ex_top5)

in_count = len(critical_label_dataset["Internal"])
in_top5 = dict(sorted(critical_label_dataset["Internal"].items(), key=lambda item: int(item[1]), reverse=True)[:10])
print("Internal: {} {:.2f}%".format(in_count, in_count/all_count*100))
print("Internal top 5:\n",in_top5)

bot_count = len(critical_label_dataset["Bot"])
bot_top5 = dict(sorted(critical_label_dataset["Bot"].items(), key=lambda item: int(item[1]), reverse=True)[:10])
print("Bot: {} {:.2f}%".format(bot_count, bot_count/all_count*100))
print("Bot top 5:\n",bot_top5)

All: 271
External: 166 61.25%
External top 5:
 {'breaking': '78', 'P3: important': '50', 'P1: urgent': '46', 'on hold': '45', '- Breaking Change': '40', 'P2: required': '40', 'Changelog/Not Needed': '34', 'Docs/Not Needed': '34', 'breaking changes': '33', 'on-hold': '25', 'P0: critical': '23', 'blocked': '19', '💻 Change: Feature': '18', 'Change: Implementation :computer:': '18', 'security': '17', 'Tests/Not Needed': '17', 'breaking change': '16', 'priority: 4 (important)': '16', 'feedback needed': '15', 'votes needed': '15', 'severity: regression': '14', 'performance': '14', 'priority: 5 (nice to have)': '14', 'cla-not-required': '13', 'BREAKING CHANGE': '13', 'Type: Security': '13', 'x⁴ ⋅ hold': '12', 'severity: 4 (inconvenient)': '12', 'severity: 3 (broken)': '12', 'oca-required': '10', 'action needed': '9', 'severity: inconvenient': '9', 'P-High': '9', 'priority: 2 (urgent)': '9', 'semver-major: breaking change': '8', 'pr: blocked': '8', 'priority: 3 (required)': '8', 'priority: hig

In [165]:
with open('attention_label_ranking.json', 'w') as outfile:
     json.dump(critical_label_dataset, outfile, indent=4)